In [1]:
## Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from pathlib import Path
import xlwings as xw

In [12]:
# Initialize an empty DataFrame for all times
df_all = pd.DataFrame(columns=["Mass", "Standoff distance", "Angle", "Arrival time", "Change time", "Trough time", "End time"])

# Mapping dictionary
mapping = {"1": 0, "2": 15, "3": 30, "4": 45, "5": 60}

In [13]:
#Access input folder
input_dir1 = Path ("G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m")
print ("1",input_dir1)

#Access folders inside input folder
input_dir2 =  [folder for folder in input_dir1.iterdir() if folder.is_dir()]
print ("2",input_dir2)

# Create output directory
output_dir = Path("G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_fr7m")
output_dir.mkdir(exist_ok = True)

# Define current work directory
current_dir = Path.cwd()

# Access the template to be used to generate files
excel_template = current_dir/ "Template.xlsx"

1 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_curves_excel7m
2 [WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m00.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m02.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m04.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m06.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m08.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m10.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m12.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m14.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m16.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m18.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m20.5kg'), WindowsPath('G:/Chamodi/LSDYNA3D/RP_TH_models/RP_curves_excel7m/07m22.5kg'), WindowsPath('G:/Cham

In [14]:
# Identify peaks with tolerance parameters
height_threshold = 2  # Minimum height of peaks to detect
prominence = 0.01  # Minimum prominence of peaks in pressure units
width = 8  # Minimum width of peaks in time units

## Determine peak before smoothing

In [15]:
for folder in input_dir2:
    
    SD = int(folder.name[0:2])
    EM = float(folder.name[3:-2])
    
    # Make a list of data file names
    #files = [file for file in folder.rglob("*")]
    files = [file for file in folder.rglob("*") if "5" not in file.stem]
    
    # Create output folders
    output_dir1 = output_dir/ folder.name
    print ("output_dir1", output_dir1)
    output_dir1.mkdir(exist_ok = True)
    
    # Initiate xlwings library
    with xw.App (visible = False) as app:
        
        # Access each file 
        for file in files:
            
            # Assign value to A based on file.stem
            file_name = file.stem
            A = mapping.get(file_name, None)
            print(file_name, A)

            # Open excel template
            wb = app.books.open(excel_template)
            
            # Read data from Excel file
            df = pd.read_excel(file)
            
            # Extract time and pressure columns
            time = df.iloc[:, 0].values
            pressure = df.iloc[:, 1].values
            pressure -= 0.18
            
            # Find the index of the peak value
            peak_index = np.argmax(pressure)
            
            pressure = pressure[peak_index:]
            time = time[peak_index:]
            
            # Find peaks
            peaks, properties = find_peaks(pressure, height=height_threshold, prominence=prominence, width=width)
            
            # Ensure there are at least two peaks
            if len(peaks) >= 1:
                #first_peak_index = peaks[0]
            
                # Create a copy of pressure data to modify
                smoothed_pressure = np.copy(pressure)
            
                # Define a window around each peak after the first peak to apply smoothing
                window_radius_r = 100
                window_radius_l = 50 # Adjust the window radius as needed
            
                for peak_index in peaks[:]:
                    start = max(peak_index - window_radius_l, 0)
                    end = min(peak_index + window_radius_r + 1, len(pressure))
            
                    # Extract values just outside the window
                    if start > 0 and end < len(pressure):
                        start_value = pressure[start - 1]
                        end_value = pressure[end]
                    elif start > 0:
                        start_value = pressure[start - 1]
                        end_value = start_value
                    elif end < len(pressure):
                        end_value = pressure[end]
                        start_value = end_value
                    else:
                        start_value = end_value = np.mean(pressure)  # Fallback in case there are no valid points
            
                    # Create a linearly spaced array between start_value and end_value
                    interpolated_values = np.linspace(start_value, end_value, end - start)
            
                    # Apply the interpolated values to the points within the window
                    smoothed_pressure[start:end] = interpolated_values
                
                # Find the index of the peak value
                #peak_index = np.argmax(smoothed_pressure)
                
                # Find the index of the lowest (most negative) value after the peak
                lowest_index = np.argmin(smoothed_pressure[:])
                
                if smoothed_pressure[lowest_index] < 0:
                    
                    # Find the first zero crossing between peak and lowest value
                    zero_crossing_index_1 = next((i for i, val in enumerate(smoothed_pressure[:lowest_index], start=0) if val <= 0), None)
                
                    # Find the first zero crossing after the lowest value
                    zero_crossing_index_2 = next((i for i, val in enumerate(smoothed_pressure[lowest_index:], start=lowest_index) if val >= 0), None)
                    
                    if zero_crossing_index_2 is None:
                        zero_crossing_index_2 = min(range(lowest_index, len(smoothed_pressure)),key=lambda i: abs(smoothed_pressure[i]))
                    
                else:                        
                        
                    zero_crossing_index_1 = lowest_index
                    zero_crossing_index_2 = lowest_index
                    
                # Find the peak (maximum) values of each array
                max_pressure = np.max(pressure)
                max_smoothed_pressure = np.max(smoothed_pressure)
                
                # Check if the peak values are the same
                if max_pressure == max_smoothed_pressure:
                
                    # Arrival, change, trough and end times
                    AT = time[0]
                    CT = time[zero_crossing_index_1]
                    TT = time[lowest_index]
                    ET = time[zero_crossing_index_2]
                    print (peak_index, zero_crossing_index_1, lowest_index, zero_crossing_index_2)
                    
                    # Append the values to DataFrames
                    # Creating a new DataFrame to append
                    row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A], "Arrival time": [AT], "Change time": [CT], "Trough time": [TT], "End time": [ET]})
                    df_all = pd.concat([df_all, row_all], ignore_index=True)
                    
                    # Create a dictionary with the arrays
                    data1 = {'Time': time,
                            'Smoothed_Pressure': smoothed_pressure,
                            'Pressure': pressure}
                    data2 = {'Adj Time': time[: zero_crossing_index_2+1],
                            'Adj Pressure': smoothed_pressure[: zero_crossing_index_2+1]}
                    
                    # Create DataFrames from dictionaries
                    df1 = pd.DataFrame(data1)
                    df2 = pd.DataFrame(data2)
                    
                    # Concatenate DataFrames along the columns
                    df_combined = pd.concat([df1, df2], axis=1)
                    
                    # Create a DataFrame from the dictionary
                    df3 = pd.DataFrame(df_combined)
                    
                    # Write dataframe in excel template
                    wb.sheets[0].range("A1").options(index=False).value = df3
                    
                    print ("file_name",file_name)
                    
                    # Create files in output directory
                    wb.save(output_dir1/f"{file_name}.xlsx")
                
                else:
                    print("The peak values are different.")
                
            else:
                print("Less than one peaks found in the data.")
                
                # Create a dictionary with the arrays
                data3 = {'Time': time,'Pressure': pressure}
                
                # Create a DataFrame from the dictionary
                df4 = pd.DataFrame(data3)
                
                # Write dataframe in excel template
                wb.sheets[0].range("A1").options(index=False).value = df4
                
                print ("file_name",file_name)
                
                # Create files in output directory
                wb.save(output_dir1/f"{file_name}.xlsx")

output_dir1 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_curves_fr7m\07m00.5kg
1 0
247 31 154 1120
file_name 1
2 15
240 75 186 1102
file_name 2
3 30
Less than one peaks found in the data.
file_name 3
4 45
Less than one peaks found in the data.
file_name 4
output_dir1 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_curves_fr7m\07m02.5kg
1 0
280 186 229 538
file_name 1
2 15
272 171 221 532
file_name 2
3 30
244 179 505 1207
file_name 3
4 45
203 263 435 977
file_name 4
output_dir1 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_curves_fr7m\07m04.5kg
1 0
304 239 597 639
file_name 1
2 15
293 210 241 556
file_name 2
3 30
258 206 530 1309
file_name 3
4 45
211 275 453 1069
file_name 4
output_dir1 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_curves_fr7m\07m06.5kg
1 0
337 468 638 678
file_name 1
2 15
324 236 272 592
file_name 2
3 30
271 291 550 1385
file_name 3
4 45
215 311 463 1137
file_name 4
output_dir1 G:\Chamodi\LSDYNA3D\RP_TH_models\RP_curves_fr7m\07m08.5kg
1 0
375 490 686 719
file_name 1
2 15
358 275 307 632
file_name 2
3 30
290 3

In [16]:
# Write dataframes in excel template
all_path = 'RP_all.xlsx'

df_all.to_excel(all_path, index=False)

## Without smoothing

In [ ]:
for folder in input_dir2:
    
    SD = int(folder.name[0:2])
    EM = float(folder.name[3:-2])
    
    # Make a list of data file names
    files = [file for file in folder.rglob("*")]
    
    # Create output folders
    output_dir1 = output_dir/ folder.name
    print ("output_dir1", output_dir1)
    output_dir1.mkdir(exist_ok = True)
    
    # Initiate xlwings library
    with xw.App (visible = False) as app:
        
        # Access each file 
        for file in files:
            
            # Assign value to A based on file.stem
            file_name = file.stem
            A = mapping.get(file_name, None)
            print(file_name, A)

            # Open excel template
            wb = app.books.open(excel_template)
            
            # Read data from Excel file
            df = pd.read_excel(file)
            
            # Extract time and pressure columns
            time = df.iloc[:, 0].values
            pressure = df.iloc[:, 1].values
             
            # Find the index of the peak value
            peak_index = np.argmax(pressure)
            
            # Filter array after peak value
            adj_pressure = pressure[peak_index:]
            adj_time = time[peak_index:]
                
            # Find the index of the lowest (most negative) value after the peak
            lowest_index = np.argmin(adj_pressure)
                
            if adj_pressure[lowest_index] < 0:
                    
                # Find the first zero crossing between peak and lowest value
                zero_crossing_index_1 = next((i for i, val in enumerate(adj_pressure[0:lowest_index], start=0) if val <= 0), None)
                
                # Find the first zero crossing after the lowest value
                zero_crossing_index_2 = next((i for i, val in enumerate(adj_pressure[lowest_index:], start=lowest_index) if val >= 0), None)
                    
                if zero_crossing_index_2 is None:
                    zero_crossing_index_2 = min(range(lowest_index, len(adj_pressure)),key=lambda i: abs(adj_pressure[i]))
                    
            else:                        
                        
                zero_crossing_index_1 = lowest_index
                zero_crossing_index_2 = lowest_index
                
            # Arrival, change, trough and end times
            AT = adj_time[0]
            CT = adj_time[zero_crossing_index_1]
            TT = adj_time[lowest_index]
            ET = adj_time[zero_crossing_index_2]
            print (peak_index, zero_crossing_index_1, lowest_index, zero_crossing_index_2)
                
            # Append the values to DataFrames
            # Creating a new DataFrame to append
            row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A], "Arrival time": [AT], "Change time": [CT], "Trough time": [TT], "End time": [ET]})
            df_all = pd.concat([df_all, row_all], ignore_index=True)
                
            # Create a dictionary with the arrays
            data1 = {'Time': time,
                    'Pressure': pressure}
            data2 = {'Adj Time': adj_time[0: zero_crossing_index_2+1],
                    'Adj Pressure': adj_pressure[0: zero_crossing_index_2+1]}
                
            # Create DataFrames from dictionaries
            df1 = pd.DataFrame(data1)
            df2 = pd.DataFrame(data2)
                
            # Concatenate DataFrames along the columns
            df_combined = pd.concat([df1, df2], axis=1)
                
            # Create a DataFrame from the dictionary
            df3 = pd.DataFrame(df_combined)
                
            # Write dataframe in excel template
            wb.sheets[0].range("A1").options(index=False).value = df3
                
            print ("file_name",file_name)
                
            # Create files in output directory
            wb.save(output_dir1/f"{file_name}.xlsx") 

# Extra

In [ ]:
for folder in input_dir2:
    
    SD = int(folder.name[0:2])
    EM = float(folder.name[3:-2])
    
    # Make a list of data file names
    files = [file for file in folder.rglob("*")]
    
    # Create output folders
    output_dir1 = output_dir/ folder.name
    print ("output_dir1", output_dir1)
    output_dir1.mkdir(exist_ok = True)
    
    # Initiate xlwings library
    with xw.App (visible = False) as app:
        
        # Access each file 
        for file in files:
            
            # Assign value to A based on file.stem
            file_name = file.stem
            A = mapping.get(file_name, None)
            print(file_name, A)

            # Open excel template
            wb = app.books.open(excel_template)
            
            # Read data from Excel file
            df = pd.read_excel(file)
            
            # Extract time and pressure columns
            time = df.iloc[:, 0].values
            pressure = df.iloc[:, 1].values
            
            # Find peaks
            peaks, properties = find_peaks(pressure, height=height_threshold, prominence=prominence, width=width)
            
            # Ensure there are at least two peaks
            if len(peaks) >= 2:
                first_peak_index = peaks[0]
            
                # Create a copy of pressure data to modify
                smoothed_pressure = np.copy(pressure)
            
                # Define a window around each peak after the first peak to apply smoothing
                window_radius_r = 20
                window_radius_l = 20 # Adjust the window radius as needed
            
                for peak_index in peaks[1:]:
                    start = max(peak_index - window_radius_l, 0)
                    end = min(peak_index + window_radius_r + 1, len(pressure))
            
                    # Extract values just outside the window
                    if start > 0 and end < len(pressure):
                        start_value = pressure[start - 1]
                        end_value = pressure[end]
                    elif start > 0:
                        start_value = pressure[start - 1]
                        end_value = start_value
                    elif end < len(pressure):
                        end_value = pressure[end]
                        start_value = end_value
                    else:
                        start_value = end_value = np.mean(pressure)  # Fallback in case there are no valid points
            
                    # Create a linearly spaced array between start_value and end_value
                    interpolated_values = np.linspace(start_value, end_value, end - start)
            
                    # Apply the interpolated values to the points within the window
                    smoothed_pressure[start:end] = interpolated_values
                
                # Find the index of the peak value
                peak_index = np.argmax(smoothed_pressure)
                
                # Find the index of the lowest (most negative) value after the peak
                lowest_index = peak_index + np.argmin(smoothed_pressure[peak_index:])
                
                if smoothed_pressure[lowest_index] < 0:
                    
                    # Find the first zero crossing between peak and lowest value
                    zero_crossing_index_1 = next((i for i, val in enumerate(smoothed_pressure[peak_index:lowest_index], start=peak_index) if val <= 0), None)
                
                    # Find the first zero crossing after the lowest value
                    zero_crossing_index_2 = next((i for i, val in enumerate(smoothed_pressure[lowest_index:], start=lowest_index) if val >= 0), None)
                    
                    if zero_crossing_index_2 is None:
                        zero_crossing_index_2 = min(range(lowest_index, len(smoothed_pressure)),key=lambda i: abs(smoothed_pressure[i]))
                    
                else:                        
                        
                    zero_crossing_index_1 = lowest_index
                    zero_crossing_index_2 = lowest_index
                
                # Arrival, change, trough and end times
                AT = time[peak_index]
                CT = time[zero_crossing_index_1]
                TT = time[lowest_index]
                ET = time[zero_crossing_index_2]
                print (peak_index, zero_crossing_index_1, lowest_index, zero_crossing_index_2)
                
                # Append the values to DataFrames
                # Creating a new DataFrame to append
                row_arrival = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A], "Arrival time": [AT]})
                df_arrival = pd.concat([df_arrival, row_arrival], ignore_index=True)
                row_change = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A], "Change time": [CT]})
                df_change = pd.concat([df_change, row_change], ignore_index=True)
                row_trough = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A], "Trough time": [TT]})
                df_trough = pd.concat([df_trough, row_trough], ignore_index=True)
                row_end = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A], "End time": [ET]})
                df_end = pd.concat([df_end, row_end], ignore_index=True)
                row_all = pd.DataFrame({"Mass": [EM], "Standoff distance": [SD], "Angle": [A], "Arrival time": [AT], "Change time": [CT], "Trough time": [TT], "End time": [ET]})
                df_all = pd.concat([df_all, row_all], ignore_index=True)
                
                # Create a dictionary with the arrays
                data1 = {'Time': time,
                        'Smoothed_Pressure': smoothed_pressure,
                        'Pressure': pressure}
                data2 = {'Adj Time': time[peak_index: zero_crossing_index_2+1],
                        'Adj Pressure': smoothed_pressure[peak_index: zero_crossing_index_2+1]}
                
                # Create DataFrames from dictionaries
                df1 = pd.DataFrame(data1)
                df2 = pd.DataFrame(data2)
                
                # Concatenate DataFrames along the columns
                df_combined = pd.concat([df1, df2], axis=1)
                
                # Create a DataFrame from the dictionary
                df3 = pd.DataFrame(df_combined)
                
                # Write dataframe in excel template
                wb.sheets[0].range("A1").options(index=False).value = df3
                
                print ("file_name",file_name)
                
                # Create files in output directory
                wb.save(output_dir1/f"{file_name}.xlsx") 
                
            else:
                print("Less than two peaks found in the data.")
                
                # Create a dictionary with the arrays
                data3 = {'Time': time,'Pressure': pressure}
                
                # Create a DataFrame from the dictionary
                df4 = pd.DataFrame(data3)
                
                # Write dataframe in excel template
                wb.sheets[0].range("A1").options(index=False).value = df4
                
                print ("file_name",file_name)
                
                # Create files in output directory
                wb.save(output_dir1/f"{file_name}.xlsx")